This is text.
weeeeeeeeeeeeeeeeee

# Imports


In [1]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import PIL
import tensorflow as tf
from tensorflow import keras
import cv2
from keras import layers
from keras.layers import Input, Conv2D, Dense, concatenate
from keras.models import Model

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


# Image loading and pre-processing

In [88]:

images = np.load("train_x.npy")

In [118]:
# function that will only the digits + some of the more intense noise
def preprocess(ar):
    ret = [] 
    for image in ar:
         cop = np.copy(image)
         cop[cop < 240] = 0
         ret.append(cop)
    return ret

In [5]:
# ONLY RUN IF YOU WANT TO SAVE PREPROCESSED IMAGES
# take all the training images, preprocess them and keep them in the train_data folder. 
for i in range(len(images)):
    processedImg = images[i]
    processedImg[processedImg < 240] = 0
    cv2.imwrite('./train_data/proc_{}.jpg'.format(i), processedImg)
    if i%50 == 0:
        print('./train_data/proc_{}.jpg'.format(i))

./data/proc_0.jpg
./data/proc_50.jpg
./data/proc_100.jpg
./data/proc_150.jpg
./data/proc_200.jpg
./data/proc_250.jpg
./data/proc_300.jpg
./data/proc_350.jpg
./data/proc_400.jpg
./data/proc_450.jpg
./data/proc_500.jpg
./data/proc_550.jpg
./data/proc_600.jpg
./data/proc_650.jpg
./data/proc_700.jpg
./data/proc_750.jpg
./data/proc_800.jpg
./data/proc_850.jpg
./data/proc_900.jpg
./data/proc_950.jpg
./data/proc_1000.jpg
./data/proc_1050.jpg
./data/proc_1100.jpg
./data/proc_1150.jpg
./data/proc_1200.jpg
./data/proc_1250.jpg
./data/proc_1300.jpg
./data/proc_1350.jpg
./data/proc_1400.jpg
./data/proc_1450.jpg
./data/proc_1500.jpg
./data/proc_1550.jpg
./data/proc_1600.jpg
./data/proc_1650.jpg
./data/proc_1700.jpg
./data/proc_1750.jpg
./data/proc_1800.jpg
./data/proc_1850.jpg
./data/proc_1900.jpg
./data/proc_1950.jpg
./data/proc_2000.jpg
./data/proc_2050.jpg
./data/proc_2100.jpg
./data/proc_2150.jpg
./data/proc_2200.jpg
./data/proc_2250.jpg
./data/proc_2300.jpg
./data/proc_2350.jpg
./data/proc_240

In [90]:
# OTHERWISE:
# preprocess all training data
preprocessed_images = preprocess(images)


In [91]:
# take a look at some of them
for i in range(5): 
    cv2.imshow("image " + str(i), preprocessed_images[i])
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Digit localization


We're using a home-brewed, handmade, artisinal bounding-box method. Since we know there are only 3 digits per image and that the noise kept in the image after pre-processing is relatively low,
we can convolute over the preprocessed image and, for every box, sum up the total pixel data over the 28x28 bounding box. We then only keep the top 3 results from the convolution.
To avoid keeping the same digit in the image more than once, when we get a larger total we skip to the next box with a 28 pixel jump.

In [105]:
size = 28 # bounding box size, follows the mnist dataset digit size

# thank you Robin for writing all this out

def process_image(img):
    img = tf.keras.preprocessing.image.img_to_array(img, data_format=None, dtype=None).astype(np.uint8)
    img = img / 255
    return img

def get_box(arr, x, y):
    if (x+size > arr.shape[0]):
        x -= arr.shape[0] - size
    if (y+size > arr.shape[1]):
        y -= arr.shape[1] - size
    return np.copy(arr[x:x+size , y:y+size]), x, y

def reset_box(arr, x, y):
    x2, y2 = min(x+size, arr.shape[0]), min(y+size, arr.shape[1])
    arr[x:x+size , y:y+size] = np.zeros((x2-x, y2-y, 1))
    return arr

def get_max(img):
    max = [0]  
    changed = False
    i, j, x, y = 0, 0, 0, 0         
    prev = 0
    while x < img.shape[0]:
        while y < img.shape[1]:
            box, a , b = get_box(img, x, y)
            tot = np.sum(box)
            if tot > np.sum(max):
                changed = True
                max = box
                i, j = a, b
            if tot <= prev or tot == 0:
                y = y+size-1
            prev = tot
            y +=1
            x += 1
        y = 0

    img = reset_box(img, i, j)
    # this is because sometimes (very rarely) a digit cannot be found, so get_busiest would not return an array of proper shape. 
    if changed == False:    # I cannot resolve the bug in question right now but as it's extremely rare it will work for now (only happens 12 times over 40 000 images)
        max = np.zeros((28,28,1))
    return img, max


def get_busiest(img):
    img, first = get_max(img)
    img, second = get_max(img)
    img, third = get_max(img)

    return [first, second, third]

# function for saving individual images
def process_all(name):
    result = []
    for i in range(10000):
        print(str(round(i / 10000 * 100, 4)) + "%", end="\r")
        img = PIL.Image.open(name + "/proc_" + str(i) + ".jpg")
        img = process_image(img)
        result.append(get_busiest(img))
    np.save(name + ".npy", np.array(result))

# process all images and save into a (N x 3 x 28 x 28 x 1) np array, where each subarray contains the 3 digits in the image found with the bounding box method
def process_array(array, save=0, name=""):
    result = []
    n = len(array)
    for i in range(n):
        if (i%25 == 0): 
            print("\r" + str(round(i/n*100, 4)) + "%", end="")
        img = array[i]
        img = process_image(img)
        digits = np.array(get_busiest(img), subok=True)
        result.append(digits)
    print("\r100%: all digits localized.")
    result = np.array(result, subok=True)
    if save == 1:
        np.save(name + ".npy", result)
    return result


def save(img, i):
    for j, box in enumerate(img):
        cv2.imwrite("./processed/im_" + str(i) + '_' + str(j) + '.jpg', box*255)

def save_all():
    result = np.load("data.npy")
    for i, img in enumerate(result):
        print(str(round(i / 40000 * 100, 4)) + "%", end="\r")
        save(img, i)

def one():
    img = PIL.Image.open("data/proc_1251.jpg")
    img = process_image(img)
    boxes = get_busiest(img)
    plt.imshow(boxes[0])
    plt.show()
    save(boxes, 1251)

In [104]:
# save result in train_digts.npy
x_train = process_array(preprocessed_images, 1, "processed_train_x")


9.5625%Bugfix teehee!
16.5%Bugfix teehee!
24.5%Bugfix teehee!
33.0%Bugfix teehee!
39.6875%Bugfix teehee!
50.8125%Bugfix teehee!
51.4375%Bugfix teehee!
54.25%Bugfix teehee!
62.875%Bugfix teehee!
80.5%Bugfix teehee!
90.0625%Bugfix teehee!
98.9375%Bugfix teehee!
100%: all digits localized.


In [107]:
## THIS IS TO FIND BUGS IN THE GET_BUSIEST FUNCTION
cv2.imshow("lol", preprocessed_images[0])
cv2.waitKey()

test = get_busiest(process_image(preprocessed_images[3838]))
#print(np.array(test, dtype = "object").shape)
for img in test:
    cv2.imshow("lol", img)
    cv2.waitKey()
#cv2.imshow("lol", preprocessed_images[3838])
#cv2.waitKey()

In [106]:
# take a look at first few
print("x_train shape: " + str(x_train.shape) + "\n")

for i in range(5):
    digits = x_train[i]
    print(digits.shape)
    for j in range(3):
        cv2.imshow("%d, %d" % (i, j), digits[j])
        cv2.waitKey()

x_train shape: (40000, 3, 28, 28, 1)

(3, 28, 28, 1)
(3, 28, 28, 1)
(3, 28, 28, 1)
(3, 28, 28, 1)
(3, 28, 28, 1)


# Max digit recognition

Use the keras functional API to build a multi-input CNN that takes each digit individually as input. Each parallel CNN is based on the typical MNIST digit recognition architetcture.
Output result of 3 cnn's into final output layer to return the max. This is a very naive solution and, as I don't have much experience with CNN's, a pretty terrible one.
Ideally the last layer would be a customized layer that takes in the top result from each parallel cnn and returns the one with the largest index in their own output layer.
This is a workaround to the fact we don't have individual labels for each digit in the image, but only the maximum, therefore training has to be done differently than the prototypical MNIST recognition.

In [108]:

# seperate x_train into 3 different datsets for each image:
X1 = []
X2 = []
X3 = []
for digits in x_train:
    X1.append(digits[0])
    X2.append(digits[1])
    X3.append(digits[2])

X1 = np.array(X1)
X2 = np.array(X2)
X3 = np.array(X3)

print("X1 shape: " + str(X1.shape))
print("X2 shape: " + str(X2.shape))
print("X3 shape: " + str(X3.shape))

# load and process training labels 
with open('train_y.csv') as f:
    reader = csv.reader(f)
    y_train = list(reader)

y_train = np.array(y_train)
y_train = y_train[1:]
y_train = np.delete(y_train, 0, 1)
y_train = keras.utils.to_categorical(y_train, 10)

print("Y_train: " + str(y_train.shape))



X1 shape: (40000, 28, 28, 1)
X2 shape: (40000, 28, 28, 1)
X3 shape: (40000, 28, 28, 1)
Y_train: (40000, 10)


In [109]:

# basic digit recognition model for each branch in final model
def single_model(mname):
    input = Input(shape=(28,28,1))
    conv1 = layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1))
    x = conv1(input)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2,2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(100, activation='relu', kernel_initializer='he_uniform')(x)
    x = layers.BatchNormalization()(x)
    output = layers.Dense(10, activation = 'softmax')(x)
    model = keras.Model(inputs = input, outputs = output, name = mname)
    return model

# make one model for each digit in image
m1 = single_model("m1")
m2 = single_model("m2")
m3 = single_model("m3")

# concatenate models into one final output layer that (with better implementation) returns the max digit
y = concatenate([m1.output, m2.output, m3.output])
output = layers.Dense(10, activation = 'softmax')(y)

#create model
model = Model(inputs = [m1.input, m2.input, m3.input], outputs = [output], name = "YEEHAW")
model.summary()


Model: "YEEHAW"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 26, 26, 32)   320         input_1[0][0]                    
_____________________________________________________________________________________________

In [110]:
# compile model
model.compile(
    loss="categorical_crossentropy",
    optimizer="Adam",
    metrics=["accuracy"],
)

In [111]:
# train model
history = model.fit([X1, X2, X3], y_train, batch_size=32, epochs = 10, validation_split=0.2)

Epoch 1/10
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmpc6cp47k5.py, line 48)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmpc6cp47k5.py, line 48)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1000/1000 [==============================] - 55s 35ms/step - loss: 1.9751 - accuracy: 0.2751 - val_loss: 1.7558 - val_accuracy: 0.3598
Epoch 2/10
1000/1000 [==============================] - 34s 34ms/step - loss: 1.6928 - accuracy: 0.3977 - val_loss: 1.5939 - val_accuracy: 0.4329
Epoch 3/10
1000/1000 [==============================] - 33s 33ms/step - loss: 1.4532 - accuracy: 0.5087 

In [119]:
# TRYING MODEL ON TEST SET

# Test data preprocessing

test_images = np.load("test_x.npy")
processed_test_images = preprocess(test_images)
test_x = process_array(processed_test_images, 1, "processed_test_x")

X1 = []
X2 = []
X3 = []
for digits in test_x:
    X1.append(digits[0])
    X2.append(digits[1])
    X3.append(digits[2])

X1 = np.array(X1)
X2 = np.array(X2)
X3 = np.array(X3)

print("X1 shape: " + str(X1.shape))
print("X2 shape: " + str(X2.shape))
print("X3 shape: " + str(X3.shape))


100%: all digits localized.
X1 shape: (10000, 28, 28, 1)
X2 shape: (10000, 28, 28, 1)
X3 shape: (10000, 28, 28, 1)


In [120]:
# test on the model!
predictions = model.predict([X1, X2, X3])

In [124]:
# To look at some predictions: will print the prediction while showing original image in test dataset. 
for i in range(5):
    pred = list(predictions[i+1000])
    print(pred.index(max(pred)))
    img = test_images[i+1000]
    two_d = (np.reshape(img, (128, 128)) * 255).astype(np.uint8)
    cv2.imshow("image " + str(i), two_d)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

8
8
8
7
8


In [69]:
# save predictions in proper csv format with ID (have to remove # from the file however)
test_pred = []
for i,pred in enumerate(predictions):
    t = []
    t.append(i)
    pred = list(pred)
    t.append(int(pred.index(max(pred))))
    test_pred.append(t)
test_pred = np.array(test_pred, dtype = "int" )
print(test_pred.shape)
np.savetxt("test_pred.csv", test_pred, delimiter=",", header="Id,Category", fmt = '%i,%i')

(40000, 2)


In [45]:
print(test_pred[100])

['99' '9']
